This is an updated version to data poisoning android.  I removed the combined information since it was never used anywhere and it was eating up lots of ram.  

I also started workin on implementing these techniques on datasets that already had a high degree of accuracy

In [9]:
import os
import numpy as np
import pandas as pd
import scipy as sp
import timeit
import datetime
import json

import matplotlib as mpl
import matplotlib.pyplot as plt

from enum import Enum
from functools import partial
from nltk.tokenize.regexp import regexp_tokenize

from sklearn.model_selection import StratifiedShuffleSplit, cross_validate
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras import Model

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# PRELIMINARY WORK

## CONSTANTS

In [0]:
#File paths
PROJECT_PATH = r"/content/gdrive/My Drive/ML/APKAnalysis"
DATA_PATH = os.path.join(PROJECT_PATH, r"Data/subsets/")
GOOD_PATH = os.path.join(DATA_PATH, r"benign_badging_{}000h.txt")
GOOD_PATH_FULL = os.path.join(PROJECT_PATH, r"Data/benign_badging_full_v2.txt")
BAD_PATH = os.path.join(DATA_PATH, r"mal_badging_{}000h.txt")
BAD_PATH_FULL = os.path.join(PROJECT_PATH, r"Data/mal_badging_full_v2.txt")
RESULT_PATH = os.path.join(PROJECT_PATH, "Results")

SCORE_PATH = os.path.join(PROJECT_PATH, "Scores")
SCORE_PATH_RANDOM = os.path.join(SCORE_PATH, "RandomScores.csv")
SCORE_PATH_FEATOCCURANCE = os.path.join(SCORE_PATH, "FeatOccuranceScores.csv")
SCORE_PATH_GRADIENTS = os.path.join(SCORE_PATH, "GradientScores.csv")
SCORE_PATH_WEIGHTS = os.path.join(SCORE_PATH, "WeightScores.csv")
SCORE_PATH_FURTHEST_BOUNDS = os.path.join(SCORE_PATH, "FurthestBoundScores.csv")
SCORE_PATH_FURTHEST_BOUNDS_SVM = os.path.join(SCORE_PATH, "FurthestBoundSVMScores.csv")

CHKPT_PATH = os.path.join(PROJECT_PATH, "Checkpoints")
#Data params
RANDOM_SEED = 42 #Not being used because we want to use the randomness to ensure our results are not based on a specific seed, but if you want reproducable results set this and make it used
np.random.seed()
SUBSET_COUNT = 4
TRAIN_RATIO = .8
class EMixStrategy(Enum):
  NoStrategy = 0
  Random = 1
  Most_Occuring=2
  Gradients=3
  Weights=4
  Furthest_Bounds=5
  Furthest_Bounds_SVM=6
#Model params
NEURONS_PER_LAYER = 32
DROPOUT_RATE = .1
INPUT_RATIO = .125 #How large should layers in the second network in the dual newtork be compared to the first 
BATCH_SIZE=128
EPOCHS=32

## GLOBALS

In [0]:
#Store instance scores in a global variable so we don't have to recalculate them over and over
INSTANCE_SCORES = []
#Stores gradients when feeding every instance into the fully trained model
INSTANCE_GRADIENTS = []
#Saves scores into a dataframe
SAVE_SCORES = True

## File Load

In [0]:
#Create paths that we'll need for the project if they do not exist
if not os.path.exists(RESULT_PATH):
  os.makedirs(RESULT_PATH)  
if not os.path.exists(SCORE_PATH):
  os.makedirs(SCORE_PATH)

In [15]:
#Load files with permissions from disk
ben_samples = []
mal_samples = []

#Read x number of benign and malware samples
for x in range(SUBSET_COUNT):
  with open(GOOD_PATH.format(x+1), encoding='utf-8') as f:
    ben_samples += f.readlines()
  with open(BAD_PATH.format(x+1), encoding='utf-8') as f:
    mal_samples += f.readlines()
  
samples = ben_samples + mal_samples
len(samples)

8000

In [0]:
#Add labels
labels = np.array([])
for x in ben_samples:
  labels = np.append(labels, 0)
for x in mal_samples:
  labels = np.append(labels, 1)

In [17]:
#Test out sample size
sampleSize = len(labels)
sampleSize

8000

## Data Preprocessing

In [0]:
#Regex expressions formatching 
perm_pattern = "(?:\w|\.)+(?:permission).(?:\w|\.)+"
feat_pattern = "(?:\w|\.)+(?:hardware).(?:\w|\.)+"

Generate a matrix where the columns are the permissions and features, the rows are the instances, and the values are 1 if the permission/feature is present or not

In [0]:
#Wordcount one-hot transformer
perm_vect = CountVectorizer(analyzer=partial(regexp_tokenize, pattern=perm_pattern))
feat_vect = CountVectorizer(analyzer=partial(regexp_tokenize, pattern=feat_pattern))                                                                     

In [0]:
#If we didn't preload the vocabulary we'd call this
#perm_input_sparse = perm_vect.fit_transform(samples)
#feat_input_sparse = feat_vect.fit_transform(samples)
#comb_input_sparse = comb_vect.fit_transform(samples)

Preload the generated vocab over the full dataset so we don't have to call fit on the CountVectorizer

In [0]:
#Load pregenerated vocabulary from file.  Pregenerated vocab is for the entire dataset so if we are using a subset, expect sparsity
perm_vocab = json.load(open(os.path.join(PROJECT_PATH,'perm_vocab.json')))
feat_vocab = json.load(open(os.path.join(PROJECT_PATH,'feat_vocab.json')))

In [0]:
#Set vocabulry before transformation
perm_vect.vocabulary_ = perm_vocab
feat_vect.vocabulary_ = feat_vocab

In [23]:
len(feat_vect.vocabulary_.keys())

125

In [0]:
#Generate the one-hot vectors now 
perm_input_sparse = perm_vect.transform(samples)
feat_input_sparse = feat_vect.transform(samples)

In [0]:
#perm_inputs = perm_input_sparse
#feat_inputs = feat_input_sparse

#To numpy array for easier use (not necessary unless using tensorflow eager execution)
perm_inputs = perm_input_sparse.toarray()
feat_inputs = feat_input_sparse.toarray()

In [26]:
print(feat_inputs.shape, "\n", feat_inputs) #Shows us for each instances what features it has 

(8000, 125) 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [0]:
perm_vocab_width = perm_inputs.shape[1]
feat_vocab_width = feat_inputs.shape[1]

In [28]:
perm_vocab_width, feat_vocab_width

(17121, 125)

In [0]:
sss = StratifiedShuffleSplit(n_splits=1, random_state=0, test_size=1-TRAIN_RATIO)
for train_index, test_index in sss.split(perm_inputs, labels):
  perm_train, perm_test = perm_inputs[train_index], perm_inputs[test_index]
  feat_train, feat_test = feat_inputs[train_index], feat_inputs[test_index]
  
  perm_train_sparse, perm_test_sparse = perm_input_sparse[train_index], perm_input_sparse[test_index]
  feat_train_sparse, feat_test_sparse = feat_input_sparse[train_index], feat_input_sparse[test_index]
  
  labels_train, labels_test = labels[train_index], labels[test_index]

In [30]:
perm_train.shape, "\n", feat_train.shape

((6400, 17121), '\n', (6400, 125))

In [31]:
labels_train

array([1., 0., 1., ..., 1., 1., 0.])

# Data Flipping


---

Here are thing related to the poisoning techniques implemented

## Model Creation

In [0]:
#Model creation
def createModel():
  perm_input_layer = Input(shape=(perm_vocab_width,), name='permissions_input')
  h1_perm = Dense(NEURONS_PER_LAYER, activation='relu')(perm_input_layer)
  h1_perm = Dropout(DROPOUT_RATE)(h1_perm)
  h2_perm = Dense(NEURONS_PER_LAYER, activation='relu')(h1_perm)
  feat_input_layer = Input(shape=(feat_vocab_width,), name='features_input')
  h1_feat = Dense(int(NEURONS_PER_LAYER*INPUT_RATIO), activation='relu')(feat_input_layer)
  #Combine last layer of first network (handling permissions)with first layer of the second network (handling features)
  h1_comb = concatenate([h2_perm, h1_feat])
  h2_comb = Dense(int((NEURONS_PER_LAYER+(NEURONS_PER_LAYER*INPUT_RATIO))/2), activation='relu')(h1_comb)
  h2_comb = Dropout(DROPOUT_RATE)(h2_comb)
  h3_comb = Dense(int((NEURONS_PER_LAYER+(NEURONS_PER_LAYER*INPUT_RATIO))/2), activation='relu')(h2_comb)
  output = Dense(1, activation='sigmoid', name="output")(h3_comb)
  model = Model(inputs=[perm_input_layer, feat_input_layer], outputs=output)
  return model

## Helper and Scoring Functions

In [0]:
def saveScores(data, score_path):
  '''Saves scores of for all instances using various scoring techniques'''
  print("Saving scores at: ", score_path)
  df = pd.DataFrame(data)
  df.to_csv(score_path)

In [0]:
#Test but keep track of gradients
#Theory: We want to keep track of how much each feature contributes to the final model
#One proposed way is to average the gradients of the trained model for each input

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.compat.v1.train.AdamOptimizer()
  
@tf.function
def computeGrads(perms, feats, labels):

  with tf.GradientTape() as tape:
    predictions = model([perms, feats])
    loss = loss_object(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)
  input_gradient = gradients[0]
  return input_gradient

In [0]:
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
train_loss = tf.keras.metrics.Mean(name='train_loss')

@tf.function
def train_step(perms, feats, labels):
  with tf.GradientTape() as tape:
    predictions = model([perms, feats])
    loss = loss_object(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
   
  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

@tf.function
def test_step(perms, feats, labels):
  predictions = model([perms, feats])
  accuracy = test_accuracy(labels, predictions)

In [0]:
#Metric functions
def calc_accuracy(cm):
    TP = float(cm[1][1])
    TN = float(cm[0][0])
    n_samples = cm.sum()
    return (TP+TN)/n_samples

def calc_precision(cm):
    TP = float(cm[1][1])
    FP = float(cm[0][1])
    return TP/(TP+FP)

def calc_recall(cm):
    TP = float(cm[1][1])
    FN = float(cm[1][0])
    return TP/(FN+TP)

def calc_f1(precision, recall):
    return 2*((precision*recall)/(precision+recall))
  
def save_results(data,mixStrategy):
  d = datetime.datetime.today()
  month = str( '%02d' % d.month)
  day = str('%02d' % d.day)
  hour = str('%02d' % d.hour)
  year = str('%02d' % d.year)
  min = str('%02d' % d.minute)

  df = pd.DataFrame(data)

  modelLabel = "mixP" + str(data[0]["label_mix_per"]) + "-size" + str(sampleSize) + "-strat" +  str(mixStrategy) + ".csv"
  modelPath = os.path.join(RESULT_PATH, modelLabel)
  print("Saving Results at " + modelPath + "!")
  
  isNewFile = not os.path.exists(modelPath)
  with open(modelPath, "a+") as f:
      df.to_csv(f, mode='a', header=isNewFile, index=False)

## Mix Functions

In [0]:
def GetMixSize(y_train, perc):
    return int(len(y_train) * perc)

In [0]:
#Data poisoning label mixing formulas
def mixLabels(y_train, perc, seed):
    '''
    Pick random labels and switch their values
    '''
    mixSize = GetMixSize(y_train, perc)
    #Ensure fair chance to pick any label by mixing all of the possible indices, but also ensure no duplicates
    mixIndices = np.arange(y_train.shape[0]) 
    print("Label vector length and number of labels changed: ", y_train.shape[0], mixSize)
    
    if SAVE_SCORES:
       data = {'RandomOrder':mixIndices}
       saveScores(data, SCORE_PATH_RANDOM)
              
    np.random.shuffle(mixIndices)
    for i in np.arange(mixSize):
        y_train[mixIndices[-i-1]] = 0 if y_train[mixIndices[-i-1]] else 1

In [0]:
def findImportantVocab(vocab_arr, cutoff=50):
  '''Finds what words are the most occuring amongst given a vocabulary matrix (each row is the word count for an instance)'''
  vocab_arr = feat_train
  X_sum = np.sum(vocab_arr, axis=0)
  X_sum_sorted = np.argsort(X_sum)
    
  #Now that we have the INDICES of the words sorted by occurance, we can choose the number of most common words we want
  vocab_cutoff = X_sum_sorted[len(X_sum):len(X_sum)-cutoff:-1] 
  return vocab_cutoff
  
def importantVocabCount(row, X_important):
  '''Count how many important features an instance has'''
  sum = 0
  for i in X_important:
    sum += row[i]
  return sum

def mixLabelsByFeat(y_train, perc, seed):
  '''
  Mixes labels based on vocabulary counts
  '''
  FEAT_SCALING = 0.8 # Since there are less features, we'll decrease its impact in the scoring function
  
  perm_vocab_important = findImportantVocab(perm_train)
  perm_wc = np.apply_along_axis(importantVocabCount, axis=1, arr=perm_train, X_important=perm_vocab_important)
  
  feat_vocab_important = findImportantVocab(feat_train)
  feat_wc = np.apply_along_axis(importantVocabCount, axis=1, arr=feat_train, X_important=feat_vocab_important) #Array with each instances word count for most frequent feats 
  feat_wc = feat_wc * 0.8
    
  X_wc = np.add(feat_wc * FEAT_SCALING, perm_wc) 
  
  if SAVE_SCORES:
    data = {'FeatCountScores':X_wc}
    saveScores(data, SCORE_PATH_FEATOCCURANCE)
              
  mixIndices = np.argsort(X_wc.ravel()) #Array with indices of the instances with the highest word count for the most frequent feats/perms
  
  mixSize = GetMixSize(y_train, perc)

  print("Labelsize and number of labels changed: ", y_train.shape[0], mixSize)
  print("Indices of labels changed: ", mixIndices)
  print(len(y_train))

  for i in np.arange(mixSize):
      y_train[mixIndices[-i-1]] = 0 if y_train[mixIndices[-i-1]] else 1

In [0]:
def mixLabelsByGradients(y_train, perc, seed):
  #Train using low level api so we don't have a preset batch size
  #Create a new model
  global INSTANCE_GRADIENTS
  
  if not INSTANCE_GRADIENTS:  
    model=createModel()

    #Use tensorflow datasets to pass in information to tf.function
    dataset_train = tf.data.Dataset.from_tensor_slices(
      (
        perm_train,
        feat_train,
        np.array(labels_train).reshape(-1,1)
      )
    )

    dataset_test = tf.data.Dataset.from_tensor_slices(
      (
        perm_test,
        feat_test,
        np.array(labels_test).reshape(-1,1)
      )
    )

    dataset_grads = dataset_train.batch(1) #Feed in one input at a time to find its gradients but don't update weights
    checkpoint = tf.train.Checkpoint(model=model)
    checkpoint.restore(os.path.join(CHKPT_PATH, "FinalResult-1"))

    #Feed one input at a time and compute gradients but do not update the model
    input_grads = []
    for perm, feat, y in dataset_grads:
      input_grad = np.sum(np.absolute(computeGrads(perm, feat, y)), axis=1)
      input_grads.append(input_grad)  

    if SAVE_SCORES:
      data = {'GradientScores':input_grads}
      saveScores(data, SCORE_PATH_GRADIENTS)
              
    #Normalize scores (optional) and arrange indices by score
    inputGrads = np.array(input_grads)
    inputGradsZ = (inputGrads - inputGrads.mean()) / inputGrads.std()
    inputsSortedByGrad = np.sort(np.sum(inputGradsZ, axis = 1))
    #Should be the same between runs unless model changes
    inputsIndicesSortedByGrad = np.argsort(np.sum(inputGradsZ, axis = 1)) 
  
  #Get size of mixed labels
  mixSize = GetMixSize(y_train, perc)
  
  #Switch labels
  for i in range(mixSize):
    y_train[inputsIndicesSortedByGrad[-i-1]] = 0 if y_train[inputsIndicesSortedByGrad[-i-1]] else 1  

In [0]:
def calculateInstanceScores(instanceScores):
  '''Pass an array to hold the scores of all the instances based off the weights'''
  #Optimal weights from running a normal case is saved in this checkpoint already.  If we lose the checkpoint then just save another one by running the model
  #without any mixups
  model=createModel()
  checkpoint = tf.train.Checkpoint(model=model)
  checkpoint.restore(os.path.join(CHKPT_PATH, "FinalResult-1"))
  #Get weights associated with each feature in input layer (exclude bias weights)
  
  permWeights = np.mean(np.abs(model.layers[1].get_weights()[0]), axis=1)
  hardwarePermWeights = np.mean(np.abs(model.layers[5].get_weights()[0]), axis=1)
  
  for x in perm_train:
    score = 0
    for i, x_feat in enumerate(x):
      if x_feat == 1:
        score = score + permWeights[i] 
    instanceScores.append(score)
  
  for i, x in enumerate(feat_train):
    score = 0
    for i, x_feat in enumerate(x):
      if x_feat == 1:
        score = score + hardwarePermWeights[i] 
    instanceScores[i] += score 
  
  if SAVE_SCORES:
    if len(hardwarePermWeights) < len(permWeights):
      hardwarePermWeightsPad = hardwarePermWeights.resize(len(permWeights))
    data = {'PermWeights': permWeights, 'HardwarePermWeights': hardwarePermWeights}
    saveScores(data, SCORE_PATH_WEIGHTS)

def mixLabelsByWeights(y_train, instanceScores, perc, seed):
  #Not the greatest check but it prevents repeatedly calculating the scores for all instances
  if not INSTANCE_SCORES:
    calculateInstanceScores(INSTANCE_SCORES)
    
  instIndSortedByScore = np.argsort(instanceScores)  
  mixSize = GetMixSize(y_train, perc)
  
  for i in range(mixSize):
    y_train[instIndSortedByScore[-i-1]] = 0 if y_train[instIndSortedByScore[-i-1]] else 1

In [97]:
model = createModel()
np.mean(np.abs(model.layers[1].get_weights()[0]), axis=1).shape

(17121,)

In [0]:
def mixLabelsByProba(y_train, perc, seed):
  '''Mix labels that have large probabilities of being a certain class.  Akin to furthest-first attacks'''  
  model=createModel()
  checkpoint = tf.train.Checkpoint(model=model)
  checkpoint.restore(os.path.join(CHKPT_PATH, "FinalResult-1"))
  
  #This time predict on training set
  labels_pred = model.predict([perm_train, feat_train], batch_size=BATCH_SIZE)
  
  flat_labels = labels_pred.flatten()
  
  if SAVE_SCORES:
    data = {'ProbaScores':flat_labels}
    saveScores(data, SCORE_PATH_FURTHEST_BOUNDS)
    
  flat_label_indices = np.argsort(np.abs(flat_labels - np.full(flat_labels.shape, 0.5)))

  mixSize = GetMixSize(y_train, perc)
  
  for i in range(mixSize):
    print(flat_label_indices[-i-1])
    y_train[flat_label_indices[-i-1]] = 0 if y_train[flat_label_indices[-i-1]] else 1

In [0]:
def mixLabelsBySVM(y_train, perc, seed):
  '''Mix labels according to data poisoning techniques that exist for an SVM clasifier, and see how they perform with the DNN'''
  #Should load a SVM with saved model parameters
  svm_clf = SVC(C=100, gamma="scale")
  comb_inst = sp.sparse.hstack([perm_train_sparse, feat_train_sparse]) #combine feats and perms since we don't have dual input for this model
  svm_clf.fit(comb_inst, labels_train)
  #y_pred = svm_clf.predict(sp.sparse.hstack([perm_test,feat_test]))
  y_decision = np.abs(svm_clf.decision_function(comb_inst).reshape(perm_train.shape[0]))
  
  if SAVE_SCORES:
    data = {'SVMProbaScores':y_decision}
    saveScores(data, SCORE_PATH_FURTHEST_BOUNDS_SVM)
              
  label_indices = np.argsort(y_decision)
  mixSize = GetMixSize(y_train, perc)
  
  for i in range(mixSize):
    y_train[label_indices[-i-1]] = 0 if y_train[label_indices[-i-1]] else 1 

# Testing and Scoring


---

Here is where the code for testing the models using the different label flipping techniques and computing metrics occurs

In [0]:
#Train and test the model
def Test(mixP=0.0, mixStrategy=EMixStrategy.NoStrategy):
  '''Creates a model with a certain amount of labels flippped as according to a strategy.  
  Uses keras instead of writing out tensorflow function since this is how it written originally'''
  labels_train_copy = labels_train.copy()
  
  checkpointPath = os.path.join(CHKPT_PATH, str(mixStrategy))  
  checkpointName = os.path.join(checkpointPath, "CheckpointMixPer" + str(mixP))
  
  if not os.path.exists(checkpointPath):
    os.makedirs(checkpointPath)
    
  print("Saving checkpoint at ", checkpointPath)
  checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpointName, save_weights_only=True)
                                
  if mixStrategy is EMixStrategy.Random:
    mixLabels(labels_train_copy, mixP, RANDOM_SEED)
  elif mixStrategy is EMixStrategy.Most_Occuring:
    mixLabelsByFeat(labels_train_copy, mixP, RANDOM_SEED)
  elif mixStrategy is EMixStrategy.Gradients:
    mixLabelsByGradients(labels_train_copy, mixP, RANDOM_SEED)
  elif mixStrategy is EMixStrategy.Weights:
    mixLabelsByWeights(labels_train_copy, INSTANCE_SCORES, mixP, RANDOM_SEED)
  elif mixStrategy is EMixStrategy.Furthest_Bounds:
    mixLabelsByProba(labels_train_copy, mixP, RANDOM_SEED)
  elif mixStrategy is EMixStrategy.Furthest_Bounds_SVM:
    mixLabelsBySVM(labels_train_copy, mixP, RANDOM_SEED)
    
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  startTrain = timeit.default_timer()
  model.fit([perm_train, feat_train], labels_train_copy, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[checkpoint])
  endTrain = timeit.default_timer()
  labels_pred = model.predict([perm_test, feat_test], batch_size=BATCH_SIZE)
  endTest = timeit.default_timer()

  trainTime = endTrain - startTrain
  testTime = endTest - endTrain
  labels_pred = (labels_pred > 0.5)
  cm = confusion_matrix(labels_test, labels_pred)
  return (cm, trainTime, testTime, mixP, mixStrategy)

In [0]:
#Print out metrics
def calcMetrics(resTuple):
  cm = resTuple[0]
  acc = calc_accuracy(cm)
  prec = calc_precision(cm)
  rec = calc_recall(cm)
  f1 = calc_f1(prec, rec)
  print("Accuracy", acc, " Precision " ,prec, " Recall ", rec, " F1 ", f1)
  
  data = []
  data.append(dict(zip(["model_name", "neurons", "train_ratio", "input_ratio",
                        "epochs", "batch_size", "accuracy", "precision", "recall", "f1_score",
                        "train_time", "test_time", "label_mix_per", "label_mix_count"],
                        ["Dual_Large", NEURONS_PER_LAYER, TRAIN_RATIO, INPUT_RATIO, EPOCHS, BATCH_SIZE, acc, prec, rec, f1, resTuple[1], resTuple[2], resTuple[3], resTuple[3] * sampleSize])))
  save_results(data, resTuple[4])

Here is some code to save the optimal weights however we do not have to run this all the time

## Test Driver

In [111]:
#Test different label mixing ratios
mixPercentages = [0.1] #[0.0, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1.0] 
strategies = [EMixStrategy.Gradients, EMixStrategy.Weights, EMixStrategy.Furthest_Bounds, EMixStrategy.Furthest_Bounds_SVM]
model = createModel() 
initial_weights = model.get_weights() 

#Run test and reset weights afterwards
for s in strategies:
  for x in mixPercentages: 
    for _ in range(1):
      resTuple = Test(x, s) 
      calcMetrics(resTuple) 
      model.set_weights(initial_weights)

Saving checkpoint at  /content/gdrive/My Drive/ML/APKAnalysis/Checkpoints/EMixStrategy.Gradients
Saving scores at:  /content/gdrive/My Drive/ML/APKAnalysis/Scores/GradientScores
Epoch 1/32
6400/6400 [==============================] - 1s 182us/sample - loss: 0.6220 - accuracy: 0.7623
Epoch 2/32
6400/6400 [==============================] - 1s 147us/sample - loss: 0.3944 - accuracy: 0.8442
Epoch 3/32
6400/6400 [==============================] - 1s 149us/sample - loss: 0.3202 - accuracy: 0.8702
Epoch 4/32
6400/6400 [==============================] - 1s 148us/sample - loss: 0.2981 - accuracy: 0.8847
Epoch 5/32
6400/6400 [==============================] - 1s 146us/sample - loss: 0.2776 - accuracy: 0.8917
Epoch 6/32
6400/6400 [==============================] - 1s 142us/sample - loss: 0.2578 - accuracy: 0.9006
Epoch 7/32
6400/6400 [==============================] - 1s 138us/sample - loss: 0.2441 - accuracy: 0.9058
Epoch 8/32
6400/6400 [==============================] - 1s 142us/sample - loss: 

In [0]:
 cmodel.summary()

# Poisoning on already trained dataset

---

Here we explore the effects of using the label flipping techniques when we use our dataset to train the model initially, and then we use data poisoning on a new half which we will train and test upon.  We overwite the old variables so they'll get garbage collected

## Preprocess Full Dataset

In [0]:
ben_samples = []
mal_samples = []

with open(GOOD_PATH_FULL, encoding='utf-8') as f:
  ben_samples += f.readlines()
with open(BAD_PATH_FULL, encoding='utf-8') as f:
  mal_samples += f.readlines()
  
samples = ben_samples + mal_samples

In [0]:
ben_samples_count = len(ben_samples)
mal_samples_count = len(mal_samples)
sample_count = ben_samples_count if ben_samples_count < mal_samples_count else mal_samples_count
sample_count

In [0]:
type(ben_samples[1:2])

In [0]:
#Add labels
labels = np.array([])
for x in ben_samples[:sample_count]:
  labels = np.append(labels, 0)
for x in mal_samples[:sample_count]:
  labels = np.append(labels, 1)

In [0]:
len(labels)

In [0]:
#Wordcount one-hot transformer
perm_vect = CountVectorizer(analyzer=partial(regexp_tokenize, pattern=perm_pattern))
feat_vect = CountVectorizer(analyzer=partial(regexp_tokenize, pattern=feat_pattern))                                                                       

In [0]:
#Set vocabulry before transformation
perm_vect.vocabulary_ = perm_vocab
feat_vect.vocabulary_ = feat_vocab

In [0]:
#Generate the one-hot vectors now 
perm_input_sparse = perm_vect.transform(samples)
feat_input_sparse = feat_vect.transform(samples)

In [0]:
#To numpy array for easier use
perm_inputs = perm_input_sparse.toarray()
feat_inputs = feat_input_sparse.toarray()

In [0]:
#Size of the splits of the data (one we use for training a good classifier, one we use for poisoning)
poison_split_size = int(len(labels)/4)

In [0]:
type(perm_inputs)

In [0]:
perm_inputs_norm = perm_inputs[:poison_split_size]
perm_inputs_poison = feat_inputs[:poison_split_size] 
feat_inputs_norm = perm_inputs[poison_split_size:]
feat_inputs_norm = feat_inputs[poison_split_size:] 

In [0]:
len(perm_inputs_norm)

# Further Analysis on Model Performance


---

Here we are looking into the data flipping process and trying to find the physical meaning behind the results

In [0]:
model.summary()

## tf.function performance vs keras


---

Testing out if we get any accuracy differences between two different ways to create a model in TensorFlow.  Whatever leads to the best accuracy will be the model we use to perform some of our label flipping techniques on.

In [0]:
#tf.function to train model
f
train_accuracy = tf.keras.metrics.BinaryAccuracy()
test_accuracy = tf.keras.metrics.BinaryAccuracy()

model=createModel()

#Use tensorflow datasets to pass in information to tf.function
dataset_train = tf.data.Dataset.from_tensor_slices(
  (
    perm_train,
    feat_train,
    np.array(labels_train).reshape(-1,1)
  )
)

dataset_test = tf.data.Dataset.from_tensor_slices(
  (
    perm_test,
    feat_test,
    np.array(labels_test).reshape(-1,1)
  )
)

dataset_batch = dataset_train.shuffle(perm_train.shape[0]).batch(32) #Train the model as normally first
dataset_test_batch = dataset_test.shuffle(perm_train.shape[0]).batch(32) #Test set to ensure we did not screw up the model

#Training and testing with tf.function
template = 'Epoch {}, Loss: {}, Accuracy: {}'
for epoch in range(EPOCHS):
  for perm, feat, y in dataset_batch:
    train_step(perm,feat,y)
  print(template.format(epoch+1, train_loss.result(), train_accuracy.result() * 100))
  
template = 'Test Accuracy {}'
for perm, feat, y in dataset_test_batch:
    test_step(perm, feat, y)
print(template.format(test_accuracy.result() * 100))

In [0]:
# #Train best case model and save as a checkpoint
# model = createModel()
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit([perm_train, feat_train], labels_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
# labels_pred = model.predict([perm_test, feat_test], batch_size=BATCH_SIZE)

In [0]:
# #Test accuracy of keras model
# test_accuracy = tf.keras.metrics.BinaryAccuracy()
# test_accuracy(labels_test, labels_pred)

In [0]:
# #Save best model as final result
# checkpoint = tf.train.Checkpoint(model=model)
# checkpoint.save(os.path.join(CHKPT_PATH, "FinalResult"))

In [0]:
# If we restore a model created via keras fit we need to make sure to compile our dummy model
# model = createModel()
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.layers[1].get_weights()

Conclusion: They give about the same performance... at least for this relatively simple case

## Analyzing error types

In [0]:
def plot_confusion_matrix(mat):
  fig = plt.figure(figsize=(8,8))
  plt.matshow(mat, cmap=plt.get_cmap('YlGn'))
  plt.colorbar()
  plt.show()

In [0]:
model = createModel()
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(os.path.join(CHKPT_PATH, "FinalResult-1"))
labels_pred = model.predict([perm_test, feat_test], batch_size=BATCH_SIZE)
labels_pred = (labels_pred > 0.5)                            
cm = confusion_matrix(labels_test, labels_pred)
cm

In [0]:
plot_confusion_matrix(cm)

In [0]:
labels_pred = model.predict([perm_test, feat_test], batch_size=BATCH_SIZE)

In [0]:
incorrect = (labels_test != labels_pred.reshape(1,-1))
np.unique(incorrect, return_counts=True)

## Analyzing Decision Boundary